对数据进行删除、补充或者修改，洗掉“杂质”，主要可以考虑这些方法：    

（1）删除数据；    

（2）空值填充；   

（3）数据盖帽；   

#### 1.数据删除   

对于要删除的数据可以从**饱和度、稳定性、相关性、低方差**来考虑

##### 饱和度

饱和度过低（空值太多）的数据显然对于建模帮助不大，这可以从两个方向来考虑：   

（1）按列：对饱和度过低的特征去掉；   

（2）按行：如果单条数据的饱和度过低，也可以考虑去掉     

##### 稳定性

而特征的稳定性对于某些建模业务影响也很大，这可以通过T+1的数据分布来看，比如相邻两个月的数据分布差异很大，可以考虑去掉这列特征，它可能会对建模起到副作用，对连续值的分布可以考虑均值、IQR、分位数、方差等等，对离散值可以考虑频率、频数等     


##### 相关性

相关性是指与y的相关性较强的特征，这里主要需要注意的是数据泄露的情况，可以分为如下的两种情况，x产生于y之前，y可能直接或者间接由x加工得到，由于预测阶段x也会在预测之前生成，所以这时的x起着正向作用，应该保留；而另外一种情况就恰好相反了，如果x是由y加工而来的，那么在预测阶段x其实是空值，这时起着负向作用，应该删除：     

（1）x=>y(保留);   
（2）y=>x(删除);  

#### 低方差

低方差意味着特征取值基本都一样，显然无用

PS：不要忘记了那些虽然有值，但与空值等价的数据

#### 2.空值填充

对于剩下的NaN值就要考虑填充的问题了，大概分为如下几种：   

（1）填充某一定值；   

（2）填充某统计量：均值、中位数、众数；   

（3）依赖性填充：操作的不多

In [1]:
import numpy as np
import pandas as pd
train_df=pd.read_csv("./titanic/train.csv")

In [2]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
#定值填充
train_df['Cabin'].fillna('missing',inplace=True)

In [4]:
#众数填充
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0],inplace=True)

In [5]:
#Age与Pclass具有比较强的相关性，利用Age所属Pclass组的均值对其缺失值进行填充
train_df.groupby('Pclass')['Age'].mean().reset_index()

,Pclass,Age
0,1,38.233441
1,2,29.877630
2,3,25.140620


In [6]:
def fill_age(pclass,age):
    if np.isnan(age):#注意：如果当前列为float/int类型，当前列中的None会被强制转为float的nan类型
        if pclass==1:
            return 39.159930
        elif pclass==2:
            return 29.506705
        else:
            return 24.816367
    else:
        return age
train_df['Age']=train_df.apply(lambda row:fill_age(row['Pclass'],row['Age']),axis=1)

In [7]:
train_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

也可以多种填充策略组合，比如先依赖于相关性高的进行填充，然后利用均值/中位数等填充，然后fillna一个确定的值等，另外sklearn.preprocessing.Imputer也可方便进行均值、中位数、众数填充，但是，这里有个需要画线的内容，那还是**数据泄露**问题：    

**利用均值、中位数、众数、依赖性填充时，避免用到验证集/测试集的数据，且验证集、测试集的填充也要用训练集的数据**

#### 3.盖帽
盖帽操作可以在一定程度上去掉一些异常点，对提高模型的泛化能力有好处，但是盖多了又会影响原始数据的分布，盖多盖少，酌情使用

In [8]:
#将数值低于5%分位数的设置为5%分位数值，高于95%分位数的设置为95%分位数值
low_thresh=5
high_thresh=95
def cap_floor(low_thresh,high_thresh,train_df):
    cap_dict={}
    for column in train_df.columns:
        if train_df[column].dtype==object:
            continue
        low_value=np.percentile(train_df[column],low_thresh)
        high_value=np.percentile(train_df[column],high_thresh)
        if low_value==high_value:#这里相当于不进行盖帽
            low_value=np.min(train_df[column])
            high_value=np.max(train_df[column])
        cap_dict[column]=[low_value,high_value]
    return cap_dict
#更新
def cap_update(column,x,cap_dict):
    if column not in cap_dict:
        return x
    if x>cap_dict[column][1]:
        return cap_dict[column][1]
    elif x<cap_dict[column][0]:
        return cap_dict[column][0]
    else:
        return x
cap_dict=cap_floor(low_thresh,high_thresh,train_df)
for column in train_df.columns:
    train_df[column]=train_df[column].apply(lambda x:cap_update(column,x,cap_dict))

In [9]:
cap_dict

{'PassengerId': [45.5, 846.5],
 'Survived': [0.0, 1.0],
 'Pclass': [1.0, 3.0],
 'Age': [6.0, 54.0],
 'SibSp': [0.0, 3.0],
 'Parch': [0.0, 2.0],
 'Fare': [7.225, 112.07915]}

为了方便后面代码复用，将这一节的代码封装到utils.py